In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# i = Consumer ID
# j = Product Category
data=pd.read_csv('train.csv')

In [3]:
import itertools as it

In [4]:
data_new=pd.DataFrame(list(it.product(list(np.arange(2000)),list(np.arange(40)),list(np.arange(48)))),columns=['i','j','t'])
data_new2=data_new.copy()
data['buy']=1
data_merge=data_new2.merge(data, how='left')
data_merge.fillna(0,inplace=True)

In [5]:
#import pickle
#data_merge=pd.read_pickle('data_merge')
price=data_merge[data_merge['buy']==1].drop_duplicates('j').loc[:,['j','price']].sort_values('j').reset_index().drop(['index','j'],axis=1)

In [6]:
#SUPER FASTTTTT
for index, row in enumerate(data_merge['j'].values):
    try:
        if(data_merge.at[index,'price']==0.0):
            data_merge.set_value(index, 'price', price.loc[row,'price']) #row, column, value
    except:
        pass

C:\Users\abandyop\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [6]:
cust_prod=pd.crosstab(data['i'], data['j'])
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(cust_prod)
pca_samples = pca.transform(cust_prod)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(cust_prod)
pca_samples = pca.transform(cust_prod)
ps = pd.DataFrame(pca_samples)
tocluster = pd.DataFrame(ps[[0,1]])
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusterer = KMeans(n_clusters=4,random_state=42).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)

data_merge['cluster']=data_merge['i'].apply(lambda x:c_preds[x])


In [7]:
data_merge_d=pd.concat([data_merge,pd.get_dummies(data_merge['j'],prefix='P'),pd.get_dummies(data_merge['cluster'],prefix='C')],axis=1)

In [8]:
data_merge_d['i'] = data_merge_d['i'].astype('category')
data_merge_d['t'] = data_merge_d['t'].astype('category')
data_merge_d['advertised'] = data_merge_d['advertised'].astype('category')
data_merge_d['buy'] = data_merge_d['buy'].astype('category')


#data_merge_d['j'] = data_merge_d['j'].astype('category')

In [9]:
X=data_merge_d.drop(['buy','j'],axis=1)
y=data_merge.loc[:,'buy']

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [11]:
log_model = LogisticRegression()

In [12]:
log_model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
data_pred=pd.DataFrame(list(it.product(list(np.arange(2000)),list(np.arange(40)),list(np.arange(1)))),columns=['i','j','t'])
data_pred['t']=50
#Inserting Price Data
for index, row in enumerate(data_pred['j'].values):
    if(row!=24): #for item 24 having  20% discount
        data_pred.set_value(index, 'price', price.loc[row,'price']) #row, column, value
    else:
        data_pred.set_value(index, 'price', price.loc[row,'price']*0.80)
data_pred['advertised']=data_pred['j'].apply(lambda x:1 if(x==24) else 0)
data_pred['cluster']=data_pred['i'].apply(lambda x:c_preds[x])

C:\Users\abandyop\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\abandyop\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [14]:
data_pred_d= pd.concat([data_pred,pd.get_dummies(data_pred['j'],prefix='P'),pd.get_dummies(data_pred['cluster'],prefix='C')],axis=1)
data_pred_d['i'] = data_pred_d['i'].astype('category')
data_pred_d['t'] = data_pred_d['t'].astype('category')
data_pred_d['advertised'] = data_pred_d['advertised'].astype('category')
#data_pred_d['j'] = data_pred_d['j'].astype('category')
data_pred_d.drop('j',axis=1,inplace=True)

In [15]:
data_pred_d.shape

(80000, 49)

In [16]:
k=log_model.predict(data_pred_d)

In [17]:
prediction=data_pred.loc[:,['i','j']]

In [18]:
prediction['prediction']=log_model.predict_proba(data_pred_d)[:,1]

In [20]:
prediction.to_csv('prediction_answer.csv',index=False)